In [59]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import csv
import codecs
from bs4 import BeautifulSoup

# Step 1 : Login to LinkedIn

In [44]:
# Open Chrome and login to Linkedin
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(3) # Pause for 3 seconds before moving on to the next step
# After each step must stop for 3-5 seconds because Linkedin will block the software users automatically 
# - normally without sleep() it will run < 1s


In [45]:
# Enter your email and password to login
# Find the email field using its XPATH ( right click the tag in file HTML in Inspect => copy => copy XPATH)

email_field = driver.find_element(By.XPATH, '//*[@id="username"]')
email_field.send_keys('your_email') # Type your email to login
sleep(3)

# Find the password field using its CSS selector ( right click the tag => copy => copy selector)
password_field = driver.find_element(By.CSS_SELECTOR, 'input[name="session_password"]')
password_field.send_keys('your_password') # Type your password to login

sleep(3)


#### THE BIG bug :
#### The Selenium "AttributeError: 'WebDriver' object has no attribute 'find_element_by_id, by_XPATH, by_name,...'"
#### will be bug because the find_element_by_id,... method has been deprecated and removed starting Selenium 4.3.0.
#### Use the 'find_element' method instead or downgrade your selenium version to 4.2.0.


In [46]:
# click on the login button
login_field = driver.find_element(By.CSS_SELECTOR, '#organic-div form button[type="submit"]')
login_field.click()

sleep(3)


# Step 2 :Search for the profile we want to crawl

In [47]:
# Click search button
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-search"]/div')
search_field.click() # Click search button

sleep(2)
#Type keyword to search
search_keyword = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
search_keyword.send_keys('Data analyst people in Hanoi')

sleep(3)
# Enter to search
search_keyword.send_keys(Keys.RETURN)



# Step 3: Scape the Urls of the profiles by using beautifulsoup

In [ ]:
# Click people button to filter the people

people_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button'))
)

people_field.click() 
sleep(2)

In [49]:
# Get Profile URL from each search result page 
def getUrl():
    # Into the page source ( Right Click => View page source) of the search results page
    source = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all the profile links on the search results page
    profiles = source.find_all('a', class_='app-aware-link') # The bug is that 
    #####
    # MUST BE : class_='app-aware-link' not class = 'search-result__result-link ember-view'
    # Right click a tag have 1 profile that contains it link  => class = 'app-aware-link
    #####

    # Extract the href attribute from each profile link
    profile_links = []
    for profile in profiles:
        profile_link = profile['href']
        # Link to the profile page must start with 'https://www.linkedin.com/in' and not duplicated
        # 'https://www.linkedin.com/in' is the link to the profile page
        if profile_link not in profile_links and profile_link.startswith('https://www.linkedin.com/in'):
            profile_links.append(profile_link)

    return profile_links
print(getUrl())

['https://www.linkedin.com/in/trang-hanh-nguyen-63b595a3?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABX6Kk8Bfcjl0WPo2tJdt4xHf1es_5HDXwI', 'https://www.linkedin.com/in/ACoAAB-2lUoB2MpJ-b0esf0-ya3obxFj5Ue8dpw?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB-2lUoB2MpJ-b0esf0-ya3obxFj5Ue8dpw', 'https://www.linkedin.com/in/ACoAACOFj2wBsrDqtNNFMqF3qltEdFfV1t1ScOg?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACOFj2wBsrDqtNNFMqF3qltEdFfV1t1ScOg', 'https://www.linkedin.com/in/phuongdo176?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABzF2MwB_sPwqLk0AiOAdlV8LeV0I9AirFU', 'https://www.linkedin.com/in/ACoAAASjErMBbJrzoymMa5g_O__Btx-So2wA4G4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAASjErMBbJrzoymMa5g_O__Btx-So2wA4G4', 'https://www.linkedin.com/in/ACoAAAZ9s8ABSN2N_lOx6L033c4y5a3Vl6LtiGE?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAZ9s8ABSN2N_lOx6L033c4y5a3Vl6LtiGE', 'https://www.linkedin.com/in/trang-ha-bui-4a74aa19b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC7hkD8BEogSLAknL

In [50]:
# Get all profile links from all pages (1,2,3,4,....,n)

def get_URL_From_All_Pages():
    number_of_pages = 1 # NUMBER OF PAGES YOU WANT TO SCRAPE
    All_URL = []
    for page in range(number_of_pages):
        URLs_one_page = getUrl()
        sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to the bottom of the page
        # The code of Javascript run in python 
        sleep(2)
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'artdeco-pagination__button--next')))
        driver.execute_script("arguments[0].click();", next_button)# Click on the 'Next' button
        All_URL = All_URL + URLs_one_page
        sleep(2)
    return All_URL
URLs_All_page =  get_URL_From_All_Pages()
print(URLs_All_page)



['https://www.linkedin.com/in/trang-hanh-nguyen-63b595a3?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABX6Kk8Bfcjl0WPo2tJdt4xHf1es_5HDXwI', 'https://www.linkedin.com/in/ACoAAB-2lUoB2MpJ-b0esf0-ya3obxFj5Ue8dpw?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB-2lUoB2MpJ-b0esf0-ya3obxFj5Ue8dpw', 'https://www.linkedin.com/in/ACoAACOFj2wBsrDqtNNFMqF3qltEdFfV1t1ScOg?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACOFj2wBsrDqtNNFMqF3qltEdFfV1t1ScOg', 'https://www.linkedin.com/in/phuongdo176?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABzF2MwB_sPwqLk0AiOAdlV8LeV0I9AirFU', 'https://www.linkedin.com/in/ACoAAASjErMBbJrzoymMa5g_O__Btx-So2wA4G4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAASjErMBbJrzoymMa5g_O__Btx-So2wA4G4', 'https://www.linkedin.com/in/ACoAAAZ9s8ABSN2N_lOx6L033c4y5a3Vl6LtiGE?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAZ9s8ABSN2N_lOx6L033c4y5a3Vl6LtiGE', 'https://www.linkedin.com/in/trang-ha-bui-4a74aa19b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC7hkD8BEogSLAknL

 # Step 4: Scape the data of LinkedIn profiles, and write the data to a .CSV file

In [60]:

with open('output.csv', 'w', encoding='utf-8', newline='') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL'] 
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames=headers)# Write the header to CSV file
    writer.writeheader()
    
    # Loop through each profile link and extract the information
    for linkedin_url in URLs_All_page:
        # Open the profile page              
        driver.get(linkedin_url)

        # Get the page source of the profile page            
        page_source = BeautifulSoup(driver.page_source, 'html.parser')

        # Find the div tag that contains the information: name, job title, location             
        info_div = page_source.find('div', class_='mt2 relative')   
         
        # Extract the name from h1 tag with class name because h1 tag is a dynamic tag so it cannot be used with id
        name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').text              
        print(name)   

        # Extract the location from div tag with class name because div tag is a dynamic tag so it cannot be used with id
        job_title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()              
        print(job_title)

        # Extract the location from span tag with class name            
        location =  info_div.find('span', class_='text-body-small inline t-black--light break-words').text.strip()              
        print(location)

        # Write the information to CSV file
        writer.writerow({headers[0]:name, headers[1]: job_title, headers[2]:location, headers[3]:linkedin_url})
        print('\n')


Trang Hanh Nguyen
Consultant at McKinsey & Company
Hanoi Capital Region


Giang H. Nguyen, ACCA
Senior, Audit & Assurance, Deloitte Vietnam
Hanoi Capital Region


Lien (Caitlyn) Hoang
Equity Research Analyst at VinaCapital
Ho Chi Minh City, Ho Chi Minh City, Vietnam


Phương Đỗ (Chloe)
Business Analyst
Hanoi, Hanoi, Vietnam


Phong Le
Finance Director at TIKI Corporation
Ho Chi Minh City, Vietnam


Duy Truong
Senior BI & Data Analytics manager at HSC
Vietnam


Trang Ha Bui
Data Analyst
Vietnam


Bùi Đoàn Chung
HR Advisor, Founder & CEO Nghề Nhân sự Việt Nam
Ho Chi Minh City, Vietnam


Nguyễn Duy Chiến
ESL English Teacher  at Vinschool
Vietnam


Trung (Rudy) Dang 💡
Product @ Amanotes | Analytics
Hanoi, Hanoi, Vietnam


Vincent Phan
New Seller Flow | Shopee
Hanoi, Hanoi, Vietnam


Ha Trang Nguyen
Senior Investment Banking Analyst at SSI Securities Corporation
Vietnam


Trang Le
Analyst at McKinsey & Company
Hanoi, Hanoi, Vietnam


Hoang Huy Le
Member Board Of Directors at Techcom Securit